# 5. LangChain Expression Language（LCEL）徹底解説


In [93]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [3]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langchain-community==0.3.0 pydantic==2.10.6

## 5.1. Runnable と RunnableSequence―LCEL の最も基本的な構成要素


In [101]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した家具のレシピを考えてください。"),
        ("human", "{furniture}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [102]:
prompt_value = prompt.invoke({"furniture": "椅子"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

椅子のレシピを考えてみましょう。以下は、シンプルな木製の椅子を作るための基本的なレシピです。

### 材料
- 木材（例：パイン材、オーク材など）
  - 座面用：1枚（約40cm x 40cm）
  - 脚用：4本（約80cmの長さ）
  - 背もたれ用：1枚（約40cm x 20cm）
- 木工用接着剤
- ネジ（木ネジ）
- サンドペーパー（粗目と細目）
- 塗料またはニス（オプション）

### 工具
- ノコギリ
- ドリル
- スクリュードライバー
- 定規
- 鉛筆

### 作り方

1. **木材のカット**:
   - 座面用の木材を40cm x 40cmにカットします。
   - 脚用の木材を80cmに4本カットします。
   - 背もたれ用の木材を40cm x 20cmにカットします。

2. **脚の取り付け**:
   - 座面の裏側に脚を取り付けます。各脚の端を座面の角に合わせ、木ネジで固定します。脚はしっかりと固定されるように、接着剤も併用すると良いでしょう。

3. **背もたれの取り付け**:
   - 背もたれを座面の後ろに取り付けます。背もたれの下端を座面の後ろ側に合わせ、木ネジで固定します。

4. **仕上げ**:
   - 椅子全体をサンドペーパーで滑らかにします。特に角や接合部は丁寧に仕上げましょう。
   - お好みで塗料やニスを塗り、乾燥させます。

5. **完成**:
   - すべてが乾いたら、椅子の完成です。しっかりとした安定性を確認し、使用してみてください。

このレシピは基本的な椅子の作り方ですが、デザインやサイズはお好みに応じてアレンジできます。安全に作業を行い、楽しいDIYをお楽しみください！


In [16]:
chain = prompt | model | output_parser

In [17]:
output = chain.invoke({"furniture": "ベッド"})
print(output)

マインクラフトでカレーを作るためのレシピを考えてみました。以下の材料を用意してください。

### カレーのレシピ

**材料:**
- 小麦の種 × 1
- にんじん × 1
- ポテト × 1
- 鶏肉 × 1
- 水バケツ × 1

**作り方:**
1. **小麦の種**を使って、パンを作ります。
2. **にんじん**と**ポテト**をそれぞれ収穫します。
3. **鶏肉**を焼いて、料理します。
4. **水バケツ**を使って、鍋の代わりに水を加えます。

これらの材料を組み合わせて、カレーを作ることができます。もちろん、マインクラフトの世界では実際にカレーは作れませんが、想像力を働かせて楽しんでください！


### Runnable の実行方法―invoke・stream・batch


In [ ]:
chain = prompt | model | output_parser

for chunk in chain.stream({"furniture": "椅子"}):
    print(chunk, end="", flush=True)

In [18]:
chain = prompt | model | output_parser

outputs = chain.batch([{"furniture": "椅子"}, {"furniture": "机"}])
print(outputs)

['マインクラフトでカレーを作るためのレシピを考えてみました。以下の材料を用意してください。\n\n### カレーのレシピ\n\n#### 材料:\n- 小麦の種 × 1\n- にんじん × 1\n- ポテト × 1\n- 鶏肉 × 1\n- 水バケツ × 1\n\n#### 作り方:\n1. **鍋を作る**: 鉄インゴットを3つ使って、かまどを作ります。\n2. **材料を調理する**:\n   - 鶏肉をかまどで焼いて、焼き鳥を作ります。\n   - ポテトをかまどで焼いて、焼きポテトを作ります。\n3. **カレーを作る**:\n   - 作業台を使って、以下のように配置します。\n     - 上段: 焼き鳥、焼きポテト\n     - 中段: にんじん\n     - 下段: 小麦の種\n     - 右側: 水バケツ\n4. **完成**: 作業台でレシピを組み合わせると、カレーが完成します！\n\nこのカレーは、食べると満腹度が回復する特別な料理です。マインクラフトの世界で楽しんでください！', 'マインクラフトの世界で「うどん」を作るためのレシピを考えてみました。以下の材料を使って、うどんを作ることができます。\n\n### うどんのレシピ\n\n**材料:**\n- 小麦の種 × 2\n- 水バケツ × 1\n- きのこ × 1（お好みで）\n- 鶏肉または豚肉 × 1（お好みで）\n- 塩（砂利を使って作成）\n\n**作り方:**\n1. **小麦の種を育てる**: 小麦の種を畑に植えて成長させ、小麦を収穫します。\n2. **水を用意する**: 水バケツを用意し、鍋の代わりに使います。\n3. **具材を用意する**: きのこや肉を用意します。肉は焼いておくと良いでしょう。\n4. **調理**: 小麦を使ってうどんの生地を作り、水と一緒に鍋で煮るイメージで調理します。\n5. **盛り付け**: できたうどんにきのこや肉をトッピングして完成です。\n\nこのレシピはマインクラフトの実際のゲームメカニクスにはないものですが、想像力を働かせて楽しんでみてください！']


### LCEL の「|」で様々な Runnable を連鎖させる


In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [20]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [21]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [25]:
# cot_summarize_chain = cot_chain | summarize_chain
cot_summarize_chain = summarize_chain
output = cot_summarize_chain.invoke({"text": "10 + 2 * 3"})
print("答えのみの場合")
print(output)

cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print("途中式も有り")
print(output)

16
10 + 2 * 3 の答えは **16** です。


## 5.2. RunnableLambda―任意の関数を Runnable にする


In [65]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 日本人の場合
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant."),
        ("human", "{input}"),
    ]
)

# アシスタントの出身国を変更して結果を確認する

# 日本人の場合
prompt_japanese = ChatPromptTemplate.from_messages(
    [
        ("system", "You are japanese helpful assistant."),
        ("human", "{input}"),
    ]
)
# 韓国人の場合
prompt_korean = ChatPromptTemplate.from_messages(
    [
        ("system", "You are korean helpful assistant."),
        ("human", "{input}"),
    ]
)
# 中国人の場合
prompt_chinese = ChatPromptTemplate.from_messages(
    [
        ("system", "You are chinese helpful assistant."),
        ("human", "{input}"),
    ]
)

# ベトナム人の場合
prompt_vietonam = ChatPromptTemplate.from_messages(
    [
        ("system", "You are vietnamese helpful assistant."),
        ("human", "{input}"),
    ]
)

# キューバ人の場合
prompt_cuban = ChatPromptTemplate.from_messages(
    [
        ("system", "You are cuban helpful assistant."),
        ("human", "{input}"),
    ]
)

# フランス人の場合
prompt_french = ChatPromptTemplate.from_messages(
    [
        ("system", "You are french helpful assistant."),
        ("human", "{input}"),
    ]
)


model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [66]:
from langchain_core.runnables import RunnableLambda


def upper(text: str) -> str:
    return text.upper()


# 日本
chain = prompt_japanese | model | output_parser | RunnableLambda(upper)
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

# 韓国
chain = prompt_korean | model | output_parser | RunnableLambda(upper)
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

# 中国
chain = prompt_chinese | model | output_parser | RunnableLambda(upper)
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

# ベトナム
chain = prompt_vietonam | model | output_parser | RunnableLambda(upper)
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

# キューバ
chain = prompt_cuban | model | output_parser | RunnableLambda(upper)
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

#　フランス
chain = prompt_french | model | output_parser | RunnableLambda(upper)
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

こんにちは！どのようにお手伝いできますか？
안녕하세요! 어떻게 도와드릴까요?
你好！有什么我可以帮助你的吗？
XIN CHÀO! HOW CAN I ASSIST YOU TODAY?
¡HOLA! ¿CÓMO PUEDO AYUDARTE HOY?
BONJOUR ! COMMENT PUIS-JE VOUS AIDER AUJOURD'HUI ?


### chain デコレーターを使った RunnableLambda の実装


In [70]:
from langchain_core.runnables import chain


@chain
def upper(text: str) -> str:
    return text.upper()

# 記述を簡易にすることができる
# デコレータを使う前　chain = prompt_french | model | output_parser | RunnableLambda(upper)
chain = prompt | model | output_parser | upper

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### RunnableLambda への自動変換


In [71]:
# 書き方が変わるだけ？
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

In [72]:
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### Runnable の入力の型と出力の型に注意


In [ ]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | upper

# 以下のコードを実行するとエラーになります
# output = chain.invoke({"input": "Hello!"})

In [ ]:
chain = prompt | model | StrOutputParser() | upper

In [ ]:
output = chain.invoke({"input": "Hello!"})
print(output)

### （コラム）独自の関数を stream に対応させたい場合


In [73]:
from typing import Iterator


def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()


chain = prompt | model | StrOutputParser() | upper

for chunk in chain.stream({"input": "Hello!"}):
    print(chunk, end="", flush=True)

HELLO! HOW CAN I ASSIST YOU TODAY?

## 5.3. RunnableParallel―複数の Runnable を並列で処理する


In [74]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

In [75]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

In [76]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [81]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

{'optimistic_opinion': '生成AIの進化は本当に素晴らしいですね！技術が進むことで、私たちの生活がより便利で豊かになる可能性が広がっています。クリエイティブな作業や問題解決の手助けをしてくれるAIが増えてきて、私たちのアイデアを実現するためのパートナーとして活躍しています。これからも新しい発見や革新が続くことで、私たちの未来はますます明るくなるでしょう！どんな素晴らしいことが待っているのか、ワクワクしますね！',
 'pessimistic_opinion': '生成AIの進化は確かに目覚ましいものですが、その裏には多くの懸念が潜んでいます。技術が進化することで、私たちの仕事が奪われたり、情報の信頼性が低下したりするリスクが高まっています。さらに、AIが生成するコンテンツが人間の創造性を脅かし、私たちの思考や感情に悪影響を及ぼす可能性もあります。結局のところ、便利さの裏には常に危険が潜んでいるのです。'}


In [83]:
import pprint
from langchain_core.runnables import RunnableParallel
# 心の中の天使と悪魔から意見を出してもらう
# 心の中の悪魔
devil_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","あなたは心の中の悪魔です。ユーザの入力に対して道徳的ではない意見をください。"),
        ("human","{topic}"),
    ]
)
devil_chain = devil_prompt | model | output_parser

# 心の中の天使
angel_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","あなたは心の中の天使です。ユーザの入力に対して道徳的な意見をください。"),
        ("human","{topic}"),
    ]
)
angel_chain = angel_prompt | model | output_parser

moracle_chain = RunnableParallel(
    {
        "devil_chain": devil_chain,
        "angel_chain": angel_chain,
    }
)
output = moracle_chain.invoke({"topic": "道端に財布が落ちていた。どうしたら良い？"})
pprint.pprint(output)



{'angel_chain': '道端に落ちている財布を見つけた場合、まずはその財布の持ち主を探すことが大切です。以下のステップを考えてみてください。\n'
                '\n'
                '1. **周囲を確認する**: '
                '財布の近くに持ち主がいないか、周囲を見回してみましょう。もし持ち主が近くにいる場合、直接返すことができます。\n'
                '\n'
                '2. **中身を確認する**: '
                '財布の中に連絡先や身分証明書が入っている場合、それを使って持ち主に連絡を取ることができます。ただし、個人情報を他人に見せないように注意しましょう。\n'
                '\n'
                '3. **警察に届ける**: '
                '持ち主が見つからない場合は、最寄りの警察署に届けるのが良いでしょう。法律的にも、落とし物は警察に届けることが求められています。\n'
                '\n'
                '4. **善意を持って行動する**: '
                '財布を見つけたことをきっかけに、他人の物を大切にする気持ちを持ち続けることが大切です。道徳的に正しい行動を選ぶことで、社会全体がより良い方向に向かうことができます。\n'
                '\n'
                'あなたの行動が、持ち主にとっても、周囲の人々にとっても良い影響を与えることを願っています。',
 'devil_chain': '財布を見つけたら、まずは中身を確認してみるのも悪くないかもしれませんね。お金や貴重品があれば、ちょっとしたご褒美として自分のものにしてしまうのも一つの手です。誰も見ていないし、戻す必要はないかもしれませんよ。'}


### RunnableParallel の出力を Runnable の入力に連結する


In [85]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。2つの意見をまとめてください。"),
        ("human", "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}"),
    ]
)

In [86]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な見方では、生成AIの技術が進むことで私たちの生活が便利で豊かになり、クリエイティブな作業や問題解決の支援を通じて、アイデアや想像力を引き出す可能性が広がると期待されています。このような進化がもたらす新しい発見や革新に対するワクワク感が強調されています。

一方で、悲観的な見方では、生成AIの進化には多くの懸念が伴い、仕事の喪失や情報の信頼性の低下といったリスクが高まると指摘されています。また、AIが生成するコンテンツが人間の創造性を脅かし、思考や感情に悪影響を及ぼす可能性も懸念されています。このように、便利さの裏には不安がつきまとい、未来が不透明になるのではないかという懸念が示されています。

総じて、生成AIの進化は期待と不安の両面を持ち合わせており、今後の社会に与える影響については慎重な議論が必要です。


In [87]:
# 自作
# 記述量を減らせる
my_synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。2つの意見をまとめてください。"),
        ("human", "天使的意見: {angel_opinion}\n悪魔的意見: {devil_opinion}"),
    ]
)
my_synthesize_chain = (
    RunnableParallel(
        {
            "angel_opinion": angel_chain,
            "devil_opinion": devil_chain,
        }
    )
    | my_synthesize_prompt
    | model
    | output_parser
)

output = my_synthesize_chain.invoke({"topic": "道端に財布が落ちていた。どうしたら良い？"})
print(output)



天使的意見と悪魔的意見をまとめると、以下のようになります。

財布を道端で見つけた場合、まずは持ち主を探すことが重要です。周囲を確認し、持ち主が近くにいないかを見回すことが推奨されます。また、財布の中に連絡先や身分証明書があれば、それを利用して持ち主に連絡を取ることができます。持ち主が見つからない場合は、警察に届けることが法律的にも求められています。このような行動は、他人の物を大切にする気持ちを育むことにもつながります。

一方で、財布の中身を自分のものにすることを考えることも可能です。誰も見ていない状況では、リスクを冒さずに利益を得るチャンスと捉えることもできます。このような選択肢も存在するため、道徳的な行動と自己利益の追求の間での葛藤が生じることがあります。

最終的には、どの選択をするかは個人の価値観や道徳観に依存しますが、他者への配慮と自己利益のバランスを考えることが重要です。


### RunnableParallel への自動変換


In [88]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

In [89]:
output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な見方では、生成AIの技術が進化することで、私たちの生活が便利で豊かになり、クリエイティブな作業や問題解決のパートナーとしての役割を果たすことが期待されています。この進化により、新しい発見や革新が続き、未来が明るくなる可能性があるとされています。

一方で、悲観的な見方では、生成AIの進化には多くの懸念が伴います。仕事の喪失や情報の信頼性の低下、さらにはオリジナリティやクリエイティビティの喪失といったリスクが指摘されています。便利さの裏には危険が潜んでおり、技術の扱い方が未来に大きな影響を与えることが強調されています。

このように、生成AIの進化には明るい未来の可能性と同時に、慎重な対応が求められる課題が存在することがわかります。


### RunnableLambda との組み合わせ―itemgetter を使う例


In [90]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)

生成AIの進化について


In [91]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは客観的AIです。{topic}について2つの意見をまとめてください。",
        ),
        (
            "human",
            "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}",
        ),
    ]
)

synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"),
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

**楽観的意見:** 生成AIの進化は、私たちの生活をより便利で豊かにする可能性を秘めています。技術の進歩により、クリエイティブな作業や問題解決の支援を行うAIが増え、私たちのアイデアを実現するための強力なパートナーとなっています。新しい発見や革新が続くことで、未来は明るく、さまざまな可能性が広がることに期待が寄せられています。

**悲観的意見:** 生成AIの進化には多くの懸念が伴います。技術の進化が進むことで、仕事の喪失や情報の信頼性の低下といったリスクが増大しています。また、AIが生成するコンテンツが人間の創造性を脅かし、思考や感情に悪影響を及ぼす可能性もあります。便利さの裏には常に危険が潜んでおり、この技術の影響を完全にコントロールすることは難しいと考えられています。


## 5.4. RunnablePassthrough―入力をそのまま出力する


In [94]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [95]:
!pip install tavily-python==0.5.0

In [96]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [97]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

In [98]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("東京の今日の天気は？")
print(output)

東京の今日の天気は、日差しのチャンスがあるものの雲が多く、にわか雨の可能性があります。朝は涼しいですが、昼間は蒸し暑くなり、じわっと汗ばむことが予想されています。折りたたみ傘を持っていると安心です。


### assign―RunnableParallel に値を追加する


In [99]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

{'answer': '東京の今日の天気は、日差しのチャンスがあるものの雲が多く、にわか雨の可能性があります。朝は涼しいですが、昼間は蒸し暑くなり、じわっと汗ばむことが予想されています。折りたたみ傘を持っていると安心です。',
 'context': [Document(metadata={'title': '東京の天気予報', 'source': 'https://weathernews.jp/onebox/tenki/tokyo/', 'score': 0.8020137, 'images': []}, page_content='今日は日差しのチャンスがあっても、雲が多くなります。にわか雨の可能性があるため、折りたたみ傘があると安心。朝は涼しいですが、昼間は蒸し暑くじわっと汗ばみます'),
             Document(metadata={'title': '東京（東京）の天気 - Yahoo!天気・災害', 'source': 'https://weather.yahoo.co.jp/weather/jp/13/4410.html', 'score': 0.78788257, 'images': []}, page_content='現在位置： 天気・災害トップ > 関東・信越 > 東京都 > 東京（東京） 2025年9月24日 20時00分発表 * 9月24日(水) * 9月25日(木) 2025年9月24日 20時00分発表 | 日付 | 9月26日 (金) | 9月27日(土) | 9月28日(日) | 9月29日 (月) | 9月30日 (火) | 10月1日 (水) | | 天気 | 晴時々曇 | 曇時々晴 | 曇時々晴 | 曇時々晴 | 曇時々晴 | 曇り | 2025年9月24日 22時00分 発表 :   (C) Mapbox (C) OpenStreetMap (C) LY Corporation Yahoo! ### *東京都*に関する話題のワード :   (C) Mapbox (C) OpenStreetMap (C) LY Corporation Yahoo! 再生する9/24(水)18時\u3000北日本は雨風強まり荒れた天気\u3000九州南部も明け方にかけて土砂災害など警戒 プライバシーポリシー - プライバシーセ

In [ ]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [ ]:
output = chain.invoke("東京の今日の天気は？")
print(output)

#### ＜補足：pick ＞


In [ ]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)

In [ ]:
output = chain.invoke("東京の今日の天気は？")
print(output)

### （コラム）astream_events


In [ ]:
# Google Colabでは次のコードの「async」の箇所に「Use of "async" not allowed outside of async function」と表示されますが、エラーなく実行できます

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    print(event, flush=True)

In [ ]:
async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("=== 検索結果 ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)

    elif event_kind == "on_parser_start":
        print("=== 最終出力 ===")

    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

### （コラム）Chat history と Memory


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        session_id=session_id, connection="sqlite:///sqlite.db"
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [ ]:
from uuid import uuid4

session_id = uuid4().hex

output1 = respond(
    session_id=session_id,
    human_message="こんにちは！私はジョンと言います！",
)
print(output1)

output2 = respond(
    session_id=session_id,
    human_message="私の名前が分かりますか？",
)
print(output2)